# Causal Attention Mechanisms

This notebook builds off the Simple Weighted Attention Mechanism notebooks. We implemented the causal attention mechanism which is just a self-attention mechanism (like the simple weighted one) where we only consider tokens that appear previously when attempting to predict the next token in the sequence.

Causal attention
> Restricts model to only consider previous and current inputs in a sequence when processing any given token when computing attention scores. 
>
> -- _Sebastian Raschka - Build a Large Language Model from Scratch_

The code is adapted again, from Build a Large Language Model from Scratch.

Import the required modules.

In [3]:
import torch
import ttnn
from torch import nn

2025-05-03 15:39:41.910 | DEBUG    | ttnn.library_tweaks:prepare_dir_as_metal_home:54 - Existing installation of 0.57.0rc60+any detected
2025-05-03 15:39:41.933 | DEBUG    | ttnn:<module>:83 - Initial ttnn.CONFIG:
Config{cache_path=/home/avgdev/.cache/ttnn,model_cache_path=/home/avgdev/.cache/ttnn/models,tmp_dir=/tmp/ttnn,enable_model_cache=false,enable_fast_runtime_mode=true,throw_exception_on_fallback=false,enable_logging=false,enable_graph_report=false,enable_detailed_buffer_report=false,enable_detailed_tensor_report=false,enable_comparison_mode=false,comparison_mode_should_raise_exception=false,comparison_mode_pcc=0.9999,root_report_path=generated/ttnn/reports,report_name=std::nullopt,std::nullopt}


Declare the familiar input, `context` representing the string `Your journey starts with one step`.

In [4]:
context = torch.tensor(
  [[0.43, 0.15, 0.89], # Your     (x^1)
   [0.55, 0.87, 0.66], # journey  (x^2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55]] # step     (x^6)
)

For optimizations relating to TTNN, we can declare the dimensions for `CoreGrid` for all optimized `linear` and `matmul` calls.

In [5]:
core_grid_x = 8
core_grid_y = 8

Let's now bring back our `ttnn` optimized `SelfAttention` class and use that to demonstrate causal attention.

In [6]:
torch.manual_seed(789)

class SelfAttention_v2(nn.Module):
  def __init__(self, d_in, d_out, device):
    super().__init__()

    self.W_query = nn.Linear(d_in, d_out , bias=False)
    self.W_key = nn.Linear(d_in, d_out, bias=False)
    self.W_value = nn.Linear(d_in, d_out, bias=False)

    self._device = device
    
    # Extract weight matrices from PyTorch layers and convert to TTNN once
    self.W_query_ttnn = ttnn.from_torch(
      self.W_query.weight, 
      dtype=ttnn.bfloat16, 
      layout=ttnn.TILE_LAYOUT, 
      device=self._device,
      memory_config=ttnn.L1_MEMORY_CONFIG
    )
    
    self.W_key_ttnn = ttnn.from_torch(
      self.W_key.weight, 
      dtype=ttnn.bfloat16, 
      layout=ttnn.TILE_LAYOUT, 
      device=self._device,
      memory_config=ttnn.L1_MEMORY_CONFIG
    )
    
    self.W_value_ttnn = ttnn.from_torch(
      self.W_value.weight, 
      dtype=ttnn.bfloat16, 
      layout=ttnn.TILE_LAYOUT, 
      device=self._device,
      memory_config=ttnn.L1_MEMORY_CONFIG
    )

    self._scaler = 1 / (d_out ** 0.5)

  def forward(self, x):
    x_ttnn = ttnn.from_torch(
      x, 
      dtype=ttnn.bfloat16, 
      layout=ttnn.TILE_LAYOUT, 
      device=device,
    )
    
    queries_ttnn = ttnn.linear(
      x_ttnn,
      self.W_query_ttnn,
      transpose_b=True,
      core_grid=ttnn.CoreGrid(y=core_grid_y, x=core_grid_x)
    )
    values_ttnn = ttnn.linear(
      x_ttnn,
      self.W_value_ttnn,
      transpose_b=True,
      core_grid=ttnn.CoreGrid(y=core_grid_y, x=core_grid_x)
    )
    keys_ttnn = ttnn.linear(
      x_ttnn,
      self.W_key_ttnn,
      transpose_b=True,
      core_grid=ttnn.CoreGrid(y=core_grid_y, x=core_grid_x)
    )

    attn_scores_ttnn = ttnn.matmul(
      queries_ttnn, 
      ttnn.permute(keys_ttnn, (1, 0)),
      core_grid=ttnn.CoreGrid(y=core_grid_y, x=core_grid_x)
    )

    attn_weights_ttnn = ttnn.softmax(
      attn_scores_ttnn * self._scaler,
      dim=-1
    )

    context_vec_ttnn = ttnn.matmul(
      attn_weights_ttnn,
      values_ttnn,
      core_grid=ttnn.CoreGrid(y=core_grid_y, x=core_grid_x)
    )

    context_vec = ttnn.to_torch(context_vec_ttnn)

    return context_vec

Define the dimensions which will be used to instantiate the SelfAttention_v2 class.

In [7]:
d = context.shape
d_in = context.shape[1]
d_out = context.shape[1]

d_in, d_out

(3, 3)

## Causal Attention TTNN Demonstration

Causal attention will result in different attention weights. In order to demonstrate this effectively, we'll create a `SelfAttention_v2` instance, but not perform the forward pass directly. Instead, we'll use the weights from query, key and values to perform a linear transformation (manually) on the given input batch. 

Once we have the linear transformations, we can find the attention scores against the queries and transposed result of keys through matrix multiplication.

We can get the attention weights by performing a softmax operation on the attention scores, scaled by the square root of the dimension of the keys. 

### Manually Compute Attention Weights

Note, at this point we are still not using causal attention, but just needing the necessary components to manually compute the attention weights.

In [8]:

device_id = 0
device = ttnn.open_device(device_id=device_id)

sa_v2 = SelfAttention_v2(d_in, d_out, device)
inputs_ttnn = ttnn.from_torch(
  context,
  dtype=ttnn.bfloat16,
  layout=ttnn.TILE_LAYOUT,
  device=device
)
queries_ttnn = ttnn.linear(
  inputs_ttnn,
  sa_v2.W_query_ttnn,
  transpose_b=True,
  core_grid=ttnn.CoreGrid(y=core_grid_y, x=core_grid_x)
)
keys_ttnn = ttnn.linear(
  inputs_ttnn,
  sa_v2.W_key_ttnn,
  transpose_b=True,
  core_grid=ttnn.CoreGrid(y=core_grid_y, x=core_grid_x)
)
attn_scores_ttnn = ttnn.matmul(
  queries_ttnn, 
  ttnn.permute(keys_ttnn, (1, 0)),
  core_grid=ttnn.CoreGrid(y=core_grid_y, x=core_grid_x)
)

attn_weights_ttnn = ttnn.softmax(
  attn_scores_ttnn * (1 / (d_out ** 0.5)),
  dim=-1
)

attn_scores = ttnn.to_torch(attn_scores_ttnn, device=device)
attn_weights = ttnn.to_torch(attn_weights_ttnn, device= device)
ttnn.close_device(device)

attn_weights, attn_weights.shape

New chip! We now have 1 chips
Chip initialization complete (found )
Chip initializing complete...
 ARC

 [4/4] DRAM

 [16/16] ETH

 CPU

Chip detection complete (found )


                 Device | INFO     | Opening user mode device driver
2025-05-03 15:39:46.524 | INFO     | SiliconDriver   - Opened PCI device 0; KMD version: 1.33.0, IOMMU: disabled

2025-05-03 15:39:46.534 | INFO     | SiliconDriver   - Opened PCI device 0; KMD version: 1.33.0, IOMMU: disabled
2025-05-03 15:39:46.537 | INFO     | SiliconDriver   - Harvesting mask for chip 0 is 0x200 (physical layout: 0x1, logical: 0x200, simulated harvesting mask: 0x0).
2025-05-03 15:39:46.537 | INFO     | SiliconDriver   - Opened PCI device 0; KMD version: 1.33.0, IOMMU: disabled
2025-05-03 15:39:46.538 | INFO     | SiliconDriver   - Detected PCI devices: [0]
2025-05-03 15:39:46.538 | INFO     | SiliconDriver   - Using local chip ids: {0} and remote chip ids {}
2025-05-03 15:39:46.561 | INFO     | SiliconDriver   - Software version 6.0.0, Ethernet FW version 6.14.0 (Device 0)
                  Metal | INFO     | Initializing device 0. Program cache is NOT enabled
                  Metal | INFO     | 

(TorchTensor([[0.1621, 0.1621, 0.1621, 0.1680, 0.1914, 0.1562],
              [0.1621, 0.1602, 0.1611, 0.1719, 0.1846, 0.1621],
              [0.1631, 0.1611, 0.1621, 0.1719, 0.1846, 0.1621],
              [0.1641, 0.1631, 0.1631, 0.1699, 0.1738, 0.1660],
              [0.1641, 0.1641, 0.1650, 0.1680, 0.1787, 0.1611],
              [0.1641, 0.1611, 0.1621, 0.1709, 0.1748, 0.1660]],
             dtype=torch.bfloat16),
 torch.Size([6, 6]))

### Computing Causal Attention with Torch

Now, let's break down this process using `torch` to understand the process better.

First lets create a mask to be applied on the attention scores we have computed using `ttnn` in the previous step. It is going to have the shape of `(context_length, context_length)`. In this case, (6, 6).

We can use `torch.tril` to create a lower triangular matrix with ones on the diagonal and zeros elsewhere.

In [9]:
context_length = attn_scores.shape[0]
mask_simple = torch.tril(torch.ones(context_length, context_length))
mask_simple

tensor([[1., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1.]])

Now we can apply the mask to the attention weights (as found in the earlier steps).

The result after applying the lower triangular mask should result in, well, a lower triangular matrix where the upper triangle is filled with zeros.

In [10]:
masked_simple = attn_weights * mask_simple
masked_simple

TorchTensor([[0.1621, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
             [0.1621, 0.1602, 0.0000, 0.0000, 0.0000, 0.0000],
             [0.1631, 0.1611, 0.1621, 0.0000, 0.0000, 0.0000],
             [0.1641, 0.1631, 0.1631, 0.1699, 0.0000, 0.0000],
             [0.1641, 0.1641, 0.1650, 0.1680, 0.1787, 0.0000],
             [0.1641, 0.1611, 0.1621, 0.1709, 0.1748, 0.1660]])

At this point, the masked attention weights are no longer normalized. We need to normalize them again so that the rows add up to 1 as close as possible.

In [11]:
row_sums = masked_simple.sum(dim=-1, keepdim=True)
masked_simple_norm = masked_simple / row_sums
masked_simple_norm

TorchTensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
             [0.5030, 0.4970, 0.0000, 0.0000, 0.0000, 0.0000],
             [0.3353, 0.3313, 0.3333, 0.0000, 0.0000, 0.0000],
             [0.2485, 0.2470, 0.2470, 0.2574, 0.0000, 0.0000],
             [0.1953, 0.1953, 0.1965, 0.2000, 0.2128, 0.0000],
             [0.1642, 0.1613, 0.1623, 0.1711, 0.1750, 0.1662]])

Alteratively we can maske the original attention scores to be -inf where the mask is 0.

In [12]:
mask = torch.triu(torch.ones(context_length, context_length), diagonal=1)
masked = attn_scores.masked_fill(mask.bool(), -torch.inf)
masked

TorchTensor([[-0.2295,    -inf,    -inf,    -inf,    -inf,    -inf],
             [-0.3105, -0.3281,    -inf,    -inf,    -inf,    -inf],
             [-0.3086, -0.3242, -0.3164,    -inf,    -inf,    -inf],
             [-0.1729, -0.1855, -0.1826, -0.0996,    -inf,    -inf],
             [-0.1338, -0.1309, -0.1235, -0.0820,  0.0620,    -inf],
             [-0.2314, -0.2520, -0.2500, -0.1328, -0.0840, -0.1963]],
            dtype=torch.bfloat16)

Applying softmax, we get more or less, the same answer as if we had done a norm on the triangular matrix.

In [13]:
attn_weights = torch.softmax(masked / d_out ** 0.5, dim=1)
attn_weights

TorchTensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
             [0.5039, 0.4980, 0.0000, 0.0000, 0.0000, 0.0000],
             [0.3359, 0.3320, 0.3340, 0.0000, 0.0000, 0.0000],
             [0.2480, 0.2461, 0.2471, 0.2598, 0.0000, 0.0000],
             [0.1943, 0.1943, 0.1953, 0.2002, 0.2168, 0.0000],
             [0.1631, 0.1611, 0.1611, 0.1719, 0.1768, 0.1660]],
            dtype=torch.bfloat16)

Later on, we can just do the similar steps but using `ttnn` when we write our `CausalAttention` class. 

### Dropout with Torch

Dropout is used to reduce the chances of overfitting data. We just remove some of the values in the masked attention weights to keep things spicy.

We will apply the dropout after computing the attention weights.

In `torch`, it is easy to apply dropout. We can just create a `nn.Dropout` module and assign the probability of the dropout. 

The example below shows where the dropout is applied. You'll see that now some elements in the matrix now become 0, while some become 2 (because we applied dropout at 0.5 probability, the 1s are rescaled

In [14]:
torch.manual_seed(123)
dropout = torch.nn.Dropout(0.5)
example = torch.ones(6, 6)
print(dropout(example))

tensor([[2., 2., 2., 2., 2., 2.],
        [0., 2., 0., 0., 0., 0.],
        [0., 0., 2., 0., 2., 0.],
        [2., 2., 0., 0., 0., 2.],
        [2., 0., 0., 0., 0., 2.],
        [0., 2., 0., 0., 0., 0.]])


Applying dropout to the attention weights does result in something more interesting:

In [15]:
dropout(attn_weights)

TorchTensor([[2.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
             [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
             [0.0000, 0.0000, 0.6680, 0.0000, 0.0000, 0.0000],
             [0.0000, 0.4922, 0.0000, 0.5195, 0.0000, 0.0000],
             [0.0000, 0.3887, 0.3906, 0.4004, 0.4336, 0.0000],
             [0.3262, 0.3223, 0.0000, 0.0000, 0.3535, 0.3320]],
            dtype=torch.bfloat16)

That's conceptually all there is to it. Now, let's see how we can do dropout with `ttnn`.

### Dropout with TTNN

With `ttnn`, we use `ttnn.experimental.dropout` to apply dropout. The API allows us to create a result tensor that has dropout applied to an input given some probability with appropriate scaling in one shot. 

In [16]:
from ttnn import TILE_LAYOUT


device_id = 0
device = ttnn.open_device(device_id=device_id)

attn_weights_ttnn = ttnn.to_device(
  ttnn.from_torch(attn_weights, dtype=ttnn.bfloat16, layout=TILE_LAYOUT),
  device
)

prob = 0.5
attn_weights_dropout_ttnn = ttnn.experimental.dropout(
  attn_weights_ttnn,
  seed=123,
  probability=prob,
  scale=1.0 / (1.0 - prob)
)

dropped = ttnn.to_torch(attn_weights_dropout_ttnn)

ttnn.close_device(device)

dropped

                  Metal | INFO     | Initializing device 0. Program cache is NOT enabled
                  Metal | INFO     | AI CLK for device 0 is:   1000 MHz
                  Metal | WARNING  | Circular buffer indices are not contiguous starting at 0. This will hurt dispatch performance. Non-contiguous indices: 2. First unused index: 1. Kernels: reader_dropout_interleaved_start_id
                  Metal | WARNING  | Circular buffer indices are not contiguous starting at 0. This will hurt dispatch performance. Non-contiguous indices: 2. First unused index: 1. Kernels: writer_dropout_interleaved_start_id, reader_dropout_interleaved_start_id, dropout_kernel
                  Metal | INFO     | Closing device 0
                  Metal | INFO     | Disabling and clearing program cache on device 0


TorchTensor([[2.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
             [1.0078, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
             [0.6719, 0.6641, 0.6680, 0.0000, 0.0000, 0.0000],
             [0.4961, 0.0000, 0.0000, 0.5195, 0.0000, 0.0000],
             [0.0000, 0.3887, 0.3906, 0.4004, 0.4336, 0.0000],
             [0.0000, 0.3223, 0.3223, 0.0000, 0.3535, 0.0000]],
            dtype=torch.bfloat16)

## Implementing Causal Attention with Torch

Putting it all together in `torch`, here is the `CausalAttention` class.

In [17]:
class CausalAttention(nn.Module):
  def __init__(self, d_in, d_out, context_length, dropout, qkv_bias=False):
    super().__init__()

    self.d_out = d_out
    
    self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
    self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
    self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
    
    self.dropout = nn.Dropout(dropout)

    self.mask = torch.triu(
      torch.ones(context_length, context_length),
      diagonal=1
    )

  def forward(self, x): 
    b, num_tokens, d_in = x.shape
    keys = self.W_query(x)
    queries = self.W_query(x)
    values = self.W_value(x)

    # transpose the last 2 dimensions while leaving the batch dimension alone.
    attn_scores = queries @ keys.transpose(1, 2)
    attn_scores.masked_fill(
      self.mask.bool()[:num_tokens, :num_tokens], -torch.inf
    )
    attn_weights = torch.softmax(
      attn_scores / keys.shape[-1]**0.5, dim=-1
    )

    attn_weights = self.dropout(attn_weights)

    context_vec = attn_weights @ values
    return context_vec


In [18]:
torch.manual_seed(123)
batch = torch.reshape(context, [1, context.shape[0], context.shape[1]])
context_length = batch.shape[1]
ca = CausalAttention(d_in, d_out, context_length, 0.5)
context_vecs = ca(batch)

context_vecs


tensor([[[ 0.2329,  0.3394, -0.1157],
         [ 0.2621,  0.4198, -0.1169],
         [ 0.2447,  0.3468, -0.0615],
         [ 0.4368,  0.6380, -0.1251],
         [ 0.2478,  0.3970, -0.1106],
         [ 0.1850,  0.3498, -0.1246]]], grad_fn=<UnsafeViewBackward0>)

## Implementing Causal Attention with TTNN

Same approach as the `ttnn` optimized version of `SelfAttention_v2`, we will create `ttnn` tensors of all the torch tensors created in the constructor. 

Then on forward pass, we apply linear transformations. Additional thing here is that now we get to use `ttnn.experimental.dropout` to apply dropout using `ttnn`.

In [27]:
MINUS_INFINITY = -1e9

class CausalAttention_ttnn(nn.Module):
  def __init__(self, d_in, d_out, context_length, dropout, device, qkv_bias=False):
    super().__init__()

    self.d_out = d_out
    
    self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
    self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
    self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
    self.dropout = nn.Dropout(dropout)
    self.mask = torch.triu(torch.ones(context_length, context_length), diagonal=1)

    self.device = device
    self.W_query_ttnn = ttnn.from_torch(
      self.W_query.weight,
      dtype=ttnn.bfloat16,
      layout=ttnn.TILE_LAYOUT,
      device=self.device,
      memory_config=ttnn.L1_MEMORY_CONFIG
    )

    self.W_key_ttnn = ttnn.from_torch(
      self.W_key.weight,
      dtype=ttnn.bfloat16,
      layout=ttnn.TILE_LAYOUT,
      device=self.device,
      memory_config=ttnn.L1_MEMORY_CONFIG
    )
    
    self.W_value_ttnn = ttnn.from_torch(
      self.W_value.weight,
      dtype=ttnn.bfloat16,
      layout=ttnn.TILE_LAYOUT,
      device=self.device,
      memory_config=ttnn.L1_MEMORY_CONFIG
    )

    self.mask_ttnn = ttnn.from_torch(
      self.mask,
      dtype=ttnn.bfloat16,
      layout=ttnn.TILE_LAYOUT,
      device=self.device,
      memory_config=ttnn.L1_MEMORY_CONFIG
    )

    self.dropout_prob = dropout

  def forward(self, x):
    b, num_tokens, d_in = x.shape

    x_ttnn = ttnn.from_torch(
      x,
      dtype=ttnn.bfloat16,
      layout=ttnn.TILE_LAYOUT,
      device=self.device
    )
    keys_ttnn = ttnn.linear(
      x_ttnn,
      self.W_key_ttnn,
      transpose_b=True,
      core_grid=ttnn.CoreGrid(y=core_grid_y, x=core_grid_x)
    )
    values_ttnn = ttnn.linear(
      x_ttnn,
      self.W_value_ttnn,
      transpose_b=True,
      core_grid=ttnn.CoreGrid(y=core_grid_y, x=core_grid_x)
    )
    queries_ttnn = ttnn.linear(
      x_ttnn,
      self.W_query_ttnn,
      transpose_b=True,
      core_grid=ttnn.CoreGrid(y=core_grid_y, x=core_grid_x)
    )

    keys_ttnn_transpose = ttnn.permute(keys_ttnn, (0, 2, 1))
    attn_scores_ttnn = ttnn.matmul(
      queries_ttnn,
      keys_ttnn_transpose,
      core_grid=ttnn.CoreGrid(y=core_grid_y, x=core_grid_x)
    )

    if self.dropout_prob > 0.0:
      inf_ttnn = ttnn.full_like(
        attn_scores_ttnn,
        MINUS_INFINITY, 
        layout=ttnn.TILE_LAYOUT
      )

      attn_scores_ttnn = ttnn.where(self.mask_ttnn[:num_tokens, :num_tokens], inf_ttnn, attn_scores_ttnn)

    attn_weights_ttnn = ttnn.softmax(attn_scores_ttnn * (1 / self.d_out ** 0.5), dim=-1)
    attn_weights_ttnn = ttnn.experimental.dropout(
      attn_weights_ttnn,
      seed=123,
      probability=self.dropout_prob,
      scale=1.0 / (1.0 - self.dropout_prob)
    )

    context_vec_ttnn = ttnn.matmul(
      attn_weights_ttnn,
      values_ttnn,
      core_grid=ttnn.CoreGrid(y=core_grid_y, x=core_grid_x)
    )

    context_vec = ttnn.to_torch(context_vec_ttnn, device=self.device)

    return context_vec


In [28]:
device_id = 0
device = ttnn.open_device(device_id=device_id)

torch.manual_seed(123)

# make a batch
batch = torch.reshape(context, [1, context.shape[0], context.shape[1]])

context_length = batch.shape[1]
ca = CausalAttention_ttnn(d_in, d_out, context_length, 0.5, device)
context_vecs = ca(batch)

ttnn.close_device(device)

context_vecs

                  Metal | INFO     | Initializing device 0. Program cache is NOT enabled
                  Metal | INFO     | AI CLK for device 0 is:   1000 MHz
                  Metal | INFO     | Closing device 0
                  Metal | INFO     | Disabling and clearing program cache on device 0


TorchTensor([[[ 0.6523,  1.0547, -0.5898],
              [ 0.2852,  0.4609, -0.2578],
              [ 0.6641,  1.0547, -0.3750],
              [ 0.2500,  0.3809, -0.1504],
              [ 0.3457,  0.6094, -0.1982],
              [ 0.2285,  0.4258, -0.1572]]], dtype=torch.bfloat16)

## Benchmark

In [21]:
from scripts.perf_timer import PerfTimer

In [30]:
torch.manual_seed(123)

t = PerfTimer()

t.start()
torch_tensors = torch.stack([torch.randn(1024, 2048) for _ in range(0, 1000)])
t.stop()

torch_tensors.shape, t.elapsed_ms()

(torch.Size([1000, 1024, 2048]), 7093.925714492798)

In [31]:
torch.manual_seed(123)
torch.set_printoptions(sci_mode=False)

t.reset()

t.start()

ca = CausalAttention(2048, 2048, 2048, 0.5)
for tensor in torch_tensors:
  batch = torch.reshape(tensor, [1, tensor.shape[0], tensor.shape[1]])
  result = ca(batch)
t.stop()

result, result.shape, t.elapsed_ms()

(tensor([[[     0.7871,      1.1834,      1.3771,  ...,     -0.4499,
                1.0692,      1.0057],
          [    -1.8585,     -2.1064,      0.5071,  ...,     -0.7326,
                0.9790,      0.1301],
          [     1.0886,      0.4733,     -2.0841,  ...,     -2.2115,
                0.1684,      0.6580],
          ...,
          [    -0.5703,     -0.4417,      1.5339,  ...,     -0.0207,
                0.5780,      0.4572],
          [    -0.0000,     -0.0000,      0.0000,  ...,      0.0000,
                0.0000,      0.0000],
          [     0.2907,      0.5297,     -0.3098,  ...,      3.2427,
               -0.9388,      1.0972]]], grad_fn=<UnsafeViewBackward0>),
 torch.Size([1, 1024, 2048]),
 58048.393964767456)

In [32]:

torch.manual_seed(123)
torch.set_printoptions(sci_mode=False)

t.reset()

device_id = 0
device = ttnn.open_device(device_id=device_id)

t.start()

ca = CausalAttention_ttnn(2048, 2048, 2048, 0.5, device)
for tensor in torch_tensors:
  batch = torch.reshape(tensor, [1, tensor.shape[0], tensor.shape[1]])
  result = ca(batch)
t.stop()

ttnn.close_device(device)

result, result.shape, t.elapsed_ms()

                  Metal | INFO     | Initializing device 0. Program cache is NOT enabled
                  Metal | INFO     | AI CLK for device 0 is:   1000 MHz
                  Metal | INFO     | Closing device 0
                  Metal | INFO     | Disabling and clearing program cache on device 0


(TorchTensor([[[ 0.7383,  1.1094,  1.2891,  ..., -0.4453,  1.0469,  0.9844],
               [ 0.2812,  0.4219,  0.4922,  ..., -0.1699,  0.3984,  0.3750],
               [ 0.2197,  0.0210, -0.3965,  ..., -1.2734,  0.6133,  0.6250],
               ...,
               [-0.0216,  0.0087, -0.0022,  ..., -0.0060,  0.0085,  0.0141],
               [-0.0459, -0.0352, -0.0222,  ...,  0.0297,  0.0199, -0.0039],
               [-0.0071, -0.0320, -0.0133,  ...,  0.0432,  0.0168,  0.0493]]],
             dtype=torch.bfloat16),
 torch.Size([1, 1024, 2048]),
 20172.93071746826)

In [33]:
ttnn.close_device(device)

                 Always | WARNING  | Attempting to synchronize Device 0 which is not initialized. Ignoring...
